In [8]:
from openai import OpenAI
from pydantic import BaseModel, Field
import datetime
import instructor

In [2]:
client = OpenAI()

class PersonBirthday(BaseModel):
    name: str
    age: int
    birthday: datetime.date

class Person(BaseModel):
    name: str
    age: int

In [3]:
schema = {
    'properties': {
        'name': {'type': 'string'},
        'age': {'type': 'integer'},
        'birthday': {'type': 'string', 'format': 'YYYY-MM-MM'},
    },
    'required': ['name', 'age'],
    'type': 'object',
}

resp = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {
            'role': 'user',
            'content': f'Extract `Jason Liu is thirty years old his birthday is yesterday` into json today'
        },
    ],
    functions=[{'name': 'Person', 'parameters': schema}],
    function_call='auto',
)

In [4]:
PersonBirthday.model_validate_json(resp.choices[0].message.function_call.arguments)

PersonBirthday(name='Jason Liu', age=30, birthday=datetime.date(2022, 1, 9))

In [5]:
PersonBirthday.model_json_schema()

{'properties': {'name': {'title': 'Name', 'type': 'string'},
  'age': {'title': 'Age', 'type': 'integer'},
  'birthday': {'format': 'date', 'title': 'Birthday', 'type': 'string'}},
 'required': ['name', 'age', 'birthday'],
 'title': 'PersonBirthday',
 'type': 'object'}

In [6]:
class Address(BaseModel):
    address: str = Field(description='The address of the person')

class PersonAddress(Person):
    """A Person with an address"""
    address: Address

In [7]:
PersonAddress.model_json_schema()

{'$defs': {'Address': {'properties': {'address': {'description': 'The address of the person',
     'title': 'Address',
     'type': 'string'}},
   'required': ['address'],
   'title': 'Address',
   'type': 'object'}},
 'description': 'A Person with an address',
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'age': {'title': 'Age', 'type': 'integer'},
  'address': {'$ref': '#/$defs/Address'}},
 'required': ['name', 'age', 'address'],
 'title': 'PersonAddress',
 'type': 'object'}

### Using instructor

In [9]:
client = instructor.patch(OpenAI(), mode=instructor.Mode.FUNCTIONS)

In [12]:
resp = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
            'role': 'user',
            'content': f"""
            Today is {datetime.date.today()}

            Extract `Jason Liu thirty years old his birthday is yesterday`
            he lives at 123 Main St, San Francisco, CA
            """
        },
    ],
    response_model=PersonAddress,
)

In [13]:
resp

PersonAddress(name='Jason Liu', age=30, address=Address(address='123 Main St, San Francisco, CA'))